This notebook goes through the pipeline for generating context for definitions, suggested proof paths, and tips and tricks (intuition, other proofs, etc)

In [1]:
%pip install unstructured==0.7.12
%pip install langchain
%pip install openai
%pip install chromadb
%pip install tiktoken

Note: you may need to restart the kernel to use updated packages.


In [1]:

import os
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.text_splitter import CharacterTextSplitter
from langchain.document_loaders import DirectoryLoader
from langchain.chat_models import ChatOpenAI
from langchain.schema import SystemMessage
from langchain.prompts import ChatPromptTemplate, HumanMessagePromptTemplate, MessagesPlaceholder
from langchain.chains.llm import LLMChain
from langchain.memory import ConversationBufferMemory, VectorStoreRetrieverMemory
from dotenv import load_dotenv

load_dotenv() 

openai_api_key = os.getenv("OPENAI_API_KEY")

The context module is given most recent proof steps

1. It stores proof steps inside the prompt as well as the full proof, and memory is also fed recent proof steps or definitions
2. Store proof steps but leave proof ambiguous, and memory is fed recent proof steps or definitons

It is called either upon user input or after chat execution, 

Implement Choice 2:


In [3]:
loader = DirectoryLoader('../doc/', glob='linear_map.pdf')
documents = loader.load()

In [6]:

# Split Text and create Embeddings and create VectorStore
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_documents(documents)
# Create embeddings
embeddings = OpenAIEmbeddings(openai_api_key=openai_api_key)
# load it into Chroma
vectorstore = Chroma.from_documents(texts, embeddings)

# qa = VectorDBQA.from_chain_type(llm=OpenAI(openai_api_key = openai_api_key), chain_type="stuff", vectorstore=vectorstore)


PydanticImportError: `BaseSettings` has been moved to the `pydantic-settings` package. See https://docs.pydantic.dev/2.3/migration/#basesettings-has-moved-to-pydantic-settings for more details.

For further information visit https://errors.pydantic.dev/2.3/u/import-error

In [14]:
retriever = vectorstore.as_retriever()
# For retrieving relevant context for definitions, proof suggestions, etc.
# NOT used for referencing direct definitions, proofs, etc. That will be done with vectorstore similarity search
memory = VectorStoreRetrieverMemory(memory_key="context",return_messages=True, retriever=retriever)

# Generate chain of prompts: recursively loop through each relevent definition, proof, or other context
#1 generate definition, intuition, and example.
#2 find terms attached to the definition, parse the definition, and repeat #1 for each term
prompt = ChatPromptTemplate.from_messages([
    SystemMessage(content="Please provide context or definitions for most uncommon words"), # The persistent system prompt
    MessagesPlaceholder(variable_name="context"), # Where the memory will be stored.
    HumanMessagePromptTemplate.from_template("{human_input}"), # Where the human input will injectd
])

llm = ChatOpenAI(temperature=0, model_name="gpt-3.5-turbo-16k", openai_api_key=openai_api_key)
llm_chain = LLMChain(llm=llm, prompt=prompt, memory=memory)
# qa = ConversationalRetrievalChain.from_llm(llm, retriever=retriever, memory=memory)

In [15]:
query = "What is a linear map"
llm_chain.run(query)

ValueError: variable chat_history should be a list of base messages, got 3.2 Deﬁnition linear map

A linear map from V to W is a function T properties:

V

W with the following

W

!

additivity T .u

v/

T u

T v for all u; v

V ;

C homogeneity T .!v/

D

C

2

!.T v/ for all !

V.

F and all v

D

2

2

Note that for linear maps we often use the notation T v as well as the more standard functional notation T .v/.

Some mathematicians use the term linear transformation, which means the same as linear map.

.V; W /

3.3 Notation

L

The set of all linear maps from V to W is denoted

.V; W /.

L

Let’s look at some examples of linear maps. Make sure you verify that

each of the functions deﬁned below is indeed a linear map:

3.4 Example linear maps

zero In addition to its other uses, we let the symbol 0 denote the function that takes each element of some vector space to the additive identity of another vector .V; W / is deﬁned by space. To be speciﬁc, 0

2 L

0v

0:
3.2 Deﬁnition line